#Instalación de librerías

In [ ]:
!pip install pandas  sacrebleu tiktoken datasets
!pip install --upgrade openai --quiet

import openai
import pandas as pd
from sacrebleu import corpus_bleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pya

In [ ]:
from datasets import Dataset
import numpy as np
import os
import tiktoken

## Carga de archivos para enviar en la consulta, dataset de enternamiento y el dataset de prueba sin las traducciones : train_dataset.txt y test_souces.txt

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving test_dataset (1).txt to test_dataset (1).txt


In [ ]:
train_path= '/content/train_dataset.txt'
test_path = '/content/test_sources.txt'
test_completed_path = '/content/test_dataset.txt'

In [ ]:
MODEL = "gpt-4o"
# Definir el tokenizador para el modelo GPT-4
tokenizer = tiktoken.encoding_for_model(MODEL)

def count_tokens(text):
    """Función para contar tokens en un texto usando tiktoken."""
    tokens = tokenizer.encode(text)
    return len(tokens)

def read_file(file_path):
    """Función para leer el contenido de un archivo."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


In [ ]:
# 1. Conteo de tokens fuera de la llamada al modelo

# Leer el archivo de prueba
test_content = read_file(test_path)

# Contar tokens en el contenido del archivo
test_tokens = count_tokens(test_content)
print(f"Tokens en el archivo de prueba: {test_tokens}")

## Leer el archivo de test con traducciones
test_complet_content = read_file(test_completed_path)

# Contar tokens en el contenido del archivo
test_complet_tokens = count_tokens(test_complet_content)
print(f"Tokens en el archivo de prueba con traducciones: {test_complet_tokens }")

# Leer el archivo de entrenamiento
train_content = read_file(train_path)

# Contar tokens en el archivo de entrenamiento
train_tokens = count_tokens(train_content)
print(f"Tokens en el archivo de entrenamiento: {train_tokens}")

Tokens en el archivo de prueba: 3091
Tokens en el archivo de prueba con traducciones: 5233
Tokens en el archivo de entrenamiento: 43298


In [ ]:
def read_file(file_path):
    """Función para leer el contenido del archivo."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def translate_and_count_tokens(model, input_text):
    """Función para enviar texto al modelo y contar tokens usados."""
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": input_text}],
            temperature=0,
            stream=True,
            stream_options={"include_usage": True}
        )

        # Procesa la salida y cuenta los tokens
        content = ""
        for chunk in response:
            if chunk.choices:
                content += chunk.choices[0].delta.content

        # Si el stream se cerró correctamente, los últimos datos incluirán la información de uso
        if 'usage' in chunk:
            print(f"Token usage: {chunk.usage}")
        else:
            print("No usage info available")

        return content

    except Exception as e:
        print(f"Error: {e}")
        return ""

## Traducciones sin envío previo del dataset de entrenamiento

In [ ]:
import openai
api_key = 'Clave_OpenAI'
client = openai.OpenAI(api_key=api_key)
MODEL = "gpt-4o"

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def translate_text(test):
    with open("completed_test_sources.txt", "w") as output_file:
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant who learns languages that you have just been entering"},
                    {"role": "user", "content": [
                        {"type": "text", "text": "please complete with translations in Spanish"},
                        {"type": "text", "text": test}
                    ]}
                ],
                temperature=0.5,
            )

            # Verifica y guarda la respuesta
            if response.choices:
                extracted_text = response.choices[0].message.content
                output_file.write(f"{extracted_text}\n\n")
            else:
                output_file.write("No response received.\n\n")

        except Exception as e:
            output_file.write(f"An error occurred while processing: {e}\n\n")

# Leer el contenido de los archivos de entrenamiento y test
test_content = read_file("test_sources.txt")

# Llama a la función para realizar la traducción
translate_text(test_content)

## Traducciones enviando previamente el dataste de entrenamiento

In [ ]:
import openai
api_key = 'Clave_OpenAI'
client = openai.OpenAI(api_key=api_key)
MODEL = "gpt-4o"

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def translate_text(examples, test):
    with open("completed_test_sources_with_train.txt", "w") as output_file:
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant who learns languages that you have just been entering"},
                    {"role": "user", "content": [
                        {"type": "text", "text": "please complete with translations"},
                        {"type": "text", "text": examples},
                        {"type": "text", "text": test}
                    ]}
                ],
                temperature=0.5,
            )

            # Verifica y guarda la respuesta
            if response.choices:
                extracted_text = response.choices[0].message.content
                output_file.write(f"{extracted_text}\n\n")
            else:
                output_file.write("No response received.\n\n")

        except Exception as e:
            output_file.write(f"An error occurred while processing: {e}\n\n")

# Lee el contenido de los archivos de entrenamiento y test
train_content = read_file("train_dataset.txt")
test_content = read_file("test_sources.txt")

# Llama a la función para realizar la traducción
translate_text(train_content, test_content)


#Evaluación de las traducciones

In [ ]:
import sacrebleu

def load_translations(file_path):
    translations = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith("target:"):
                # Extrae el texto después de "target:" y quita espacios y comillas
                translation = line.split("target:", 1)[1].strip().strip("'")
                translations.append(translation)
    return translations

def clean_translations(translations):
    return [t.strip("'") for t in translations]

# Cargar las traducciones del modelo y las referencias
model_translations = load_translations("completed_test_sources.txt")
model_translations_with_train = load_translations("completed_test_sources_with_train.txt")
reference_translations = [load_translations("test_dataset.txt")]  # Las referencias deben ser una lista de listas para sacrebleu
# Calcular el score SacreBLEU
bleu_score = sacrebleu.corpus_bleu(model_translations, reference_translations)
print(f"SacreBLEU score: {bleu_score.score}")
bleu_score = sacrebleu.corpus_bleu(model_translations_with_train, reference_translations)
print(f"SacreBLEU with train score: {bleu_score.score}")
'''


Con temperature 0:
SacreBLEU score: 25.14531263132354
SacreBLEU with train score: 29.57642290956401


Con temperature 0.5:
SacreBLEU score: 26.03962010287146
SacreBLEU with train score: 31.029365354664073

temperature 0.7
SacreBLEU score: 24.97737058250025
SacreBLEU with train score: 29.252665972066076

con temperature 0.8
SacreBLEU score: 26.28803002322718
SacreBLEU with train score: 28.65448865098154


temperature 1.0:
SacreBLEU score: 24.086772498679917
SacreBLEU with train score: 25.02056505769791



'''



SacreBLEU score: 27.044730862263105
SacreBLEU with train score: 31.237860248126175


'\nCon temperature 0.5:\ncon las comillas .. 28.99120053309069\nSacreBLEU score: 26.03962010287146\nSacreBLEU with train score: 31.029365354664073\n\nCon temperature 0:\nSacreBLEU score: 25.14531263132354\nSacreBLEU with train score: 29.57642290956401\n\ncon temperature 0.8 pero ejecutando antes con train:\nSacreBLEU score: 25.749454617412212\nSacreBLEU with train score: 23.15523487829909\n\ncon temperature 0.8\nSacreBLEU score: 26.28803002322718\nSacreBLEU with train score: 28.65448865098154\n\nSacreBLEU score: 22.56662950496349\nSacreBLEU with train score: 27.00434561808753\n\n\ntemperature 1.0:\nSacreBLEU score: 24.086772498679917\nSacreBLEU with train score: 25.02056505769791\n\ntemperature 0.5 8:50\n\nSacreBLEU score: 23.383724229098057\nSacreBLEU with train score: 30.061177459903313\n\ntemperature 0.7 \nSacreBLEU score: 24.97737058250025\nSacreBLEU with train score: 29.252665972066076\n'